## Introduction
The original scope of this project was limited to the prediction of wildfires in California only. This document briefly outlines the second phase of the project in which the data pipeline was scaled up more than 10 fold to include the entire continental US.

The original California only dataset contains 3.6 million lines and occupies 1.2 GB on disk. The scaled dataset contains 64 million lines and occupies 15 GB on disk.

Two main tools were applied to scale the pipeline, 1) the Python multiprocessing library was used to parallelize the processing of input files and 2) a PySpark cluster was used to accomplish manipulations too large to fit in single system memory.

## Scaled data pipeline
<p  style="text-align: center;"><b>Figure 1. Scaled data pipeline schematic</b></p>

![Scaled data pipeline schematic](https://github.com/gperdrizet/wildfire/blob/master/figures/data_analysis_flow.jpeg?raw=true)

The scaled data pipeline consists of three sequential phases. First, weather and fire data were downloaded independently from their respective sources using the Python requests library. In the second stage, Python Multiprocessing and GeoPandas libraries were used to clean, geospatialy filter and prepare the data. Lastly a PySpark cluster was employed to combine the weather and fire data into a single dataframe. The individual python scripts and Juypter Notebooks used for each individual step are linked below.

**Data acquisition - Requests**
1. [NOAA weather data](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/get_noaa_weather_data.py) - Downloads weather data files for features of interest for all of North America, 1992-2015.
2. [USDA wildfire data](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/parse_USDA_wildfire_data.ipynb) - Extracts location and time information for US wildfires from master sqlite file.

**Data cleaning, geospatial filtering and preparation - Multiprocessing/GeoPandas**
1. [Clean and convert raw NOAA weather data](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/convert_clean_raw_noaa_data.py) - Cleans up missing data and unnecessary columns in raw NOAA weather data. Also, runs a coarse geospatial filter to discard data outside of a rectangular bounding box encompassing the continental US. Saves resulting files to parquet.
2. [Geospatially filter NOAA weather data](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/fine_geospatial_filter_noaa_weather_data.py) - Discard data points which are outside the continental US landmass.
3. [Combine data by weather feature](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/collect_noaa_weather_data.py) - Collects all years of each feature into a single parquet file.
4. [Regrid fire data](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/regrid_USDA_wildfire_data.ipynb) - Maps each wildfire onto the nearest geospatial bin from the weather data.

**Combine - PysPark**
1. [Combine weather features](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/spark_combine_NOAA_weather_data.py) - Takes individual parquet files containing data for each weather feature and combines them.
3. [Preprocess weather data](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/spark_select_features.py) - Sets data types and extracts weather features of interest.
2. [Join fire and weather data](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/spark_combine_fire_and_weather_data.py) - Performs a join to combine weather and fire data into a single dataframe.
3. [Add features](https://github.com/gperdrizet/wildfire_production/blob/master/training_data_pipeline/US_all/spark_add_features.py) - Smooths data with a rolling window average and add min., max and average weather features.

## PySpark cluster resources

<p  style="text-align: center;"><b>Figure 2. Interesting Time Gang - Linux laptop cluster</b></p>

![Example of three weather variables showing the effect of adding min, max and mean](https://github.com/gperdrizet/wildfire/blob/master/figures/interesting_times_gang.jpeg?raw=true)

Interesting Times Gang is a custom linux laptop cluster built from scavenged and second hand hardware. The six nodes are headless quad-core laptops each with 8 GB memory. They boot from the headnode via a dedicated LAN. The headnode itself is a dual socket Xeon server with 16 core and 48 GB memory. The headnode is also equipped with a 8 GB Nvidia GTX1070 graphics card.

Pyspark was run on the cluster in standalone mode with 11 dual-core, 8GB workers for a total of 22 cores and 88 GB memory.

<p  style="text-align: center;"><b>Figure 3. Spark cluster configuration</b></p>

![Example of three weather variables showing the effect of smoothing via daily average](https://github.com/gperdrizet/wildfire/blob/master/figures/spark_cluster.jpeg?raw=true)